<a href="https://colab.research.google.com/github/scnossen1/sybrand_INFO5731_Spring2020/blob/master/Assignment_11_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv
import pymysql
import pandas as pd
import openpyxl
import xlrd


"""Part 1"""


filename = "ch02-data.csv"


def add_record(id, data_to_add):

    sql = "insert into ch02_data(id,dates,numbers) values(%s,%s,%s)"
    values = (id, data_to_add[0], data_to_add[1])

    dbcursor.execute(sql, values)
    db.commit()


def import_csv():
    id = 1
    data = []

    try:
        with open(filename) as f:
            reader = csv.reader(f)

            db = pymysql.connect("localhost", "root", "1234", "pydb")
            dbcursor = db.cursor()

            for row in reader:
                if reader.line_num > 1:
                    add_record(id, row)
                    id += 1

        dbcursor.close()
        db.close()

    except csv.Error as e:
        print("Error reading SCV file at line %s:%s"%(reader.line_num, e))
        exit(-1)


def get_sum():
    db = pymysql.connect("localhost", "root", "1234", "pydb")
    dbcursor = db.cursor()

    sql = "select MONTHNAME(dates),DAY(dates),SUM(numbers) from ch02_data GROUP BY MONTH(dates),DAY(dates)"

    dbcursor.execute(sql)
    results = dbcursor.fetchall()

    dbcursor.close()
    db.close()

    print("Month \t Day \t Total")
    print("-----------------------")
    for row in results:
        print(str(row[0]) + "\t" + str(row[1]) + "\t" + str(row[2]))
        print("-----------------------")

    return(results)


def exprot_to_excel(to_be_exported):
    a = [0] * len(to_be_exported)
    b = [0] * len(to_be_exported)
    c = [0] * len(to_be_exported)

    for i in range(len(to_be_exported)):
        a[i] = to_be_exported[i][0]
        b[i] = to_be_exported[i][1]
        c[i] = to_be_exported[i][2]

    dit = {'Month':a, 'Day':b, 'Total':c}

    file_path = 'export.xlsx'
    writer = pd.ExcelWriter(file_path)

    df = pd.DataFrame(dit)
    df.to_excel(writer, columns=['Month',"Day","Total"], index=False,encoding='utf-8',sheet_name='Sheet1')
    writer.save()




# exprot_to_excel(get_sum())


"""Part 2"""


def get_dataset():
    file = "catering_sale_all.xls"

    wb = xlrd.open_workbook(filename=file)
    ws = wb.sheet_by_name("Sheet1")

    dataset = []

    for r in range(ws.nrows):
        col = []
        for c in range(ws.ncols):
            col.append(ws.cell(r,c).value)
        dataset.append(col)

    return dataset


def import_xls(dataset):

    i = 8

    print(dataset)

    for i in range(len(dataset) - 1):
        sliced_dataset = dataset[i+1][1:]
        maxpos = sliced_dataset.index(max(sliced_dataset))
        print("Largest sale on", dataset[i+1][0], "was", sliced_dataset[maxpos], "of", dataset[0][maxpos+1])